# This is an example of notebook file
All code is replicated from other files in the project.

## Prepare Environment

Next console commands ensure that the environment has correct packages versions installed.

⚠️ Skip this cell if you run in local environment!

In [ ]:
!pip uninstall google-adk google-genai -y
!pip install google-adk==1.18.0 google-genai=1.49.0

Then confirm that the installation was successful by checking the versions of the installed packages.

In [1]:
import google.adk
import google.genai
print(f"Google ADK: {google.adk.__version__}, Google GenAI: {google.genai.__version__}")
# output should be "1.18.0 1.49.0"

Google ADK: 1.18.0, Google GenAI: 1.49.0


Next we should ensure that GOOGLE_API_KEY environment variable is set.

Run one of the following cells depending on service where notebook is executed. Don't run anything, if you are running notebook locally and have already set the environment variable.

In [ ]:
# Kaggle

import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

In [ ]:
# Google Colab

import os
from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"

## Execute Solution Code

Next code cell duplicated right from utils/adk_utils.py:

In [3]:
from google.genai import types

def build_retry_options() -> types.HttpRetryOptions:
    return types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

And then we can duplicate parts from flows/guidence.py

In [6]:
from google.genai import types
from google.adk.agents import Agent, LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

APP_NAME = "EXAMPLE_APP"
USER_ID = "user_123"

USER_INPUTS = [
    "Hi, my name is Dima, my pet is laying on back and I don't know what to do",
    "Ok, by the way, what is my name?"
]

# Define the agent with specific model and instructions
example_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=build_retry_options()),
    name="ExampleAgent",
    instruction="Answer user questions about pets."
)

# Initialize session service and runner
session_service = InMemorySessionService()

# Create the runner
runner = Runner(
    agent=example_agent,
    app_name=APP_NAME,
    session_service=session_service)

# Create a new session
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)
        
print("Agent: Please describe your issue with a pet:\n")
for user_input in USER_INPUTS:
    # Get user inputs
    print(f"User: {user_input}\n")
    query = types.Content(role="user", parts=[types.Part(text=user_input)])

    # Run the agent and stream responses
    async for event in runner.run_async(
        new_message=query,
        session_id=session.id,
        user_id=USER_ID):
        if event.is_final_response() and event.content and event.content.parts:
            print(f"Agent: {event.content.parts[0].text}\n")

Agent: Please describe your issue with a pet:

User: Hi, my name is Dima, my pet is laying on back and I don't know what to do

Agent: Hi Dima, it's understandable to be concerned when your pet is acting unusually! To help me figure out what might be going on, could you tell me a bit more about your pet and what they are doing?

For example, could you tell me:

*   **What kind of pet do you have?** (e.g., dog, cat, rabbit, bird, etc.)
*   **What is their breed and age?** (If you know)
*   **What does "laying on their back" look like specifically?** Are they relaxed and sleeping, or do they look uncomfortable, stiff, or in pain?
*   **Are they showing any other symptoms?** For example:
    *   Vomiting or diarrhea?
    *   Difficulty breathing?
    *   Lethargy or unusual stillness?
    *   Panting excessively?
    *   Unusual vocalizations?
    *   Any visible injuries?
*   **How long has your pet been in this position?**
*   **Did anything happen before this started?** For example, di